# Web Analytics HW3
## TwitterAPI

In [11]:
from TwitterAPI import TwitterAPI
import pandas as pd
# initialize TwitterAPI and necessary keys and tokens
CONSUMER_KEY = 'fzrSWX4oRD0OEOFnKQYDGfPSD' 
CONSUMER_SECRET = 'vV9gU8RT8cLNnufsg36CVYl7Qy40WlcHOgtbza3Chlxd48yTZp'
ACCESS_TOKEN_KEY = '891155597344813056-11oAOCuIQxNDhR7sWhb42th35h30XHr'
ACCESS_TOKEN_SECRET = 'u3SH0UvczuBxgPas5P1ghWivJm7hVsq2eZV5hhKnCwpCu'

api = TwitterAPI(
    CONSUMER_KEY,
    CONSUMER_SECRET,
    ACCESS_TOKEN_KEY,
    ACCESS_TOKEN_SECRET)

In [2]:
def crawl_function(company_list):
    count = 100
    lang = 'en'
    id_list = []
    for i in company_list:
        search_term = i
        r = api.request('search/tweets',{'q':search_term,'count':count,'lang':lang})
        with open (i+'.txt','a') as file1:
            for item in r:
                try:
                    if item['id'] in id_list:
                        pass
                    # in order to save to csv.file, I exclued tweets which with '\r' or '\n' 
                    # because pandas.to_csv will mistake these data
                    elif '\n' in str(item['text']) or '\r' in str(item['text']):
                        pass
                    else:
                        id_list.append(item['id'])
                        file1.write(str(item['user']['screen_name'])+'::'+str(item['user']['location'])+'::'+str(item['text'])+'::'+str(item['retweet_count'])+'\n')

                except UnicodeEncodeError:
                        pass
def txt2csv(company_list):
    crawled_df = pd.DataFrame(columns = ['username','location','tweet_text','count_of_retweets','company'])
    for i in company_list:
        temp = pd.read_csv((i+'.txt'),sep = '::',header = None, engine = 'python')
        temp.loc[:,'company'] = i
        temp.columns = ['username','location','tweet_text','count_of_retweets','company']
        crawled_df = pd.concat([crawled_df,temp])
    return crawled_df

In [52]:
companies = ['Amazon','Costco','ebay']
# run crawl_function 2 times
crawl_function(companies)
# convert txt to csv
df = txt2csv(companies)
# save to csv file
df.to_csv('C:\Users\youch\Desktop\crawled_byTwitterAPI.csv')

## Tweepy
### extra credit

In [8]:
import tweepy

In [55]:
def crawl_tweepy(company_list):
    """use tweepy cursor to crawl tweet data
       clean \r and \n in order to save to csv file"""
    # initialize tweepy api
    auth = tweepy.OAuthHandler(CONSUMER_KEY,CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN_KEY,ACCESS_TOKEN_SECRET)
    api = tweepy.API(auth)
    for i in company_list:
        with open (i+'.txt','a') as file2:
            for tweet in tweepy.Cursor(api.search,q=i,result_type="recent",lang="en").items(100):
                if '\n' in str(tweet.text.encode('UTF-8')) or '\r' in str(tweet.text.encode('UTF-8')):
                    pass
                else:
                    name = tweet.user.screen_name.encode('UTF-8')
                    location = tweet.user.location.encode('UTF-8')
                    tweet_content = tweet.text.encode('UTF-8')
                    retweet = tweet.retweet_count
                    file2.write(str(name)+'::'+str(location)+'::'+str(tweet_content)+'::'+str(retweet)+'\n')            

In [51]:
companies = ['Amazon','Costco','ebay']
# run two times
crawl_tweepy(companies)
# convert and save to csv file
df_tweepy = txt2csv(companies)
df_tweepy.to_csv('C:\Users\youch\Desktop\crawled_byTweepy.csv')